In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import time
import os
from os import listdir
from os.path import isfile, join

import yfinance as yf

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# make plotly work in jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

from plotly.subplots import make_subplots

import warnings
warnings.simplefilter("ignore")

In [6]:
# Company data using y-finance

def get_info_on_stock(ticker):
    stock = yf.Ticker(ticker)
    return stock

In [8]:
infosys = get_info_on_stock('INFY.BO')     # Infosys data from Bombay Stock Exchange
print(infosys.info)

{'ebitdaMargins': None, 'profitMargins': None, 'grossMargins': None, 'operatingCashflow': None, 'revenueGrowth': None, 'operatingMargins': None, 'ebitda': None, 'targetLowPrice': None, 'recommendationKey': 'none', 'grossProfits': 4077000000, 'freeCashflow': None, 'targetMedianPrice': None, 'currentPrice': 1424.2, 'earningsGrowth': None, 'currentRatio': None, 'returnOnAssets': None, 'numberOfAnalystOpinions': None, 'targetMeanPrice': None, 'debtToEquity': None, 'returnOnEquity': None, 'targetHighPrice': None, 'totalCash': None, 'totalDebt': None, 'totalRevenue': None, 'totalCashPerShare': None, 'financialCurrency': 'USD', 'maxAge': 1, 'revenuePerShare': None, 'quickRatio': None, 'recommendationMean': None, 'exchange': 'BSE', 'shortName': 'INFOSYS LTD.', 'longName': 'Infosys Limited', 'exchangeTimezoneName': 'Asia/Kolkata', 'exchangeTimezoneShortName': 'IST', 'isEsgPopulated': False, 'gmtOffSetMilliseconds': '19800000', 'quoteType': 'EQUITY', 'symbol': 'INFY.BO', 'market': 'in_market', '

In [10]:
# Historic data all the way back to 1995!

historic_data_infy = infosys.history(period="max")["Close"]
print(historic_data_infy.head())

Date
1995-05-11    0.625815
1995-05-19         NaN
1995-06-01    0.658752
1995-06-02    0.658752
1995-06-05    0.658752
Name: Close, dtype: float64


In [11]:
# Information related to stakeholders is present. 

print(infosys.major_holders)

                                              Holder    Shares Date Reported  \
0  Vanguard International Stock Index-Total Intl ...  60824139  Jul 31, 2020   
1  Vanguard International Stock Index-Emerging Ma...  48765557  Jul 31, 2020   
2             iShares Core MSCI Emerging Markets ETF  25565166  Jul 31, 2020   
3                             iShares MSCI India ETF  20102474  Aug 31, 2020   
4         Price (T.Rowe) Emerging Markets Stock Fund  12773421  Sep 30, 2020   
5                  iShares MSCI Emerging Markets ETF  12491922  Aug 31, 2020   
6         DFA Emerging Markets Core Equity Portfolio   9935138  Jul 31, 2020   
7       Price (T.Rowe) Emerging Markets Equity Trust   7381218  Sep 30, 2020   
8         Aberdeen Standard SICAV I-Indian Equity Fd   6688957  Aug 31, 2020   
9  Advisors Inner Circle Fund III-GQG Partners Em...   6484889  Jul 31, 2020   

   % Out        Value  
0  1.43%  58771324308  
1  1.14%  47119719451  
2  0.60%  24702341647  
3  0.47%  18665147109  

In [12]:
# Information related to financials is present. 

print(infosys.financials)

                                           2020-03-31     2019-03-31  \
Research Development                             None           None   
Effect Of Accounting Charges                     None           None   
Income Before Tax                        3095000000.0   3003000000.0   
Minority Interest                          55000000.0      9000000.0   
Net Income                               2331000000.0   2199000000.0   
Selling General Administrative           1351000000.0   1281000000.0   
Gross Profit                             4077000000.0   3978000000.0   
Ebit                                     2726000000.0   2697000000.0   
Operating Income                         2726000000.0   2697000000.0   
Other Operating Expenses                          NaN  13120000000.0   
Interest Expense                          -24000000.0    -24000000.0   
Extraordinary Items                              None           None   
Non Recurring                                    None           

In [13]:
# Lists to keep track of missing data
stocks_not_downloaded = []
missing_stocks = []

# Function that returns a stock dataframe from a CSV

def get_stock_df_from_csv(folder, ticker):
    try:
        df = pd.read_csv(folder + ticker + '.csv')
    except FileNotFoundError:
        print("File not found.")
    else:
        return df
    
# Function that returns a named column data from a csv

def get_column_from_csv(file, col_name):
    try:
        df = pd.read_csv(file)
    except FileNotFoundError:
        print("File not found.")
    else:
        return df[col_name]
    


In [17]:
# Download historic data of stocks using y-finance

# Step 1: Get the list of stocks to be downloaded
path = '/Users/iamsagar/Desktop/Trading Project/data/instruments/BSE_stocks.csv'
tickers = get_column_from_csv(path, 'Ticker')

# Step 2: Save stock data to csv
def save_to_csv_from_yahoo(folder, ticker):
    stock = yf.Ticker(ticker)
    try:
        print("Get data for: ", ticker)
        df = stock.history(period="5y")
        time.sleep(5)
        if df.empty:
            stocks_not_downloaded.append(ticker)
        file = folder + ticker.replace(".", "_") + ".csv"
        print(file, " saved.")
        df.to_csv(file)
    except Exception as e:
        stocks_not_downloaded.append(ticker)
        print("Couldn't download data for: ", ticker)
        
# Step 3: Pull past 5 years of data for first 20 stocks
folder = '/Users/iamsagar/Desktop/Trading Project/data/historical_all/'
for x in range(20):
    save_to_csv_from_yahoo(folder, tickers[x])
print("Finished!")

Get data for:  ABB.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/ABB_BO.csv  saved.
Get data for:  AEGISLOG.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/AEGISLOG_BO.csv  saved.
Get data for:  AMARAJABAT.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/AMARAJABAT_BO.csv  saved.
Get data for:  AMBALALSA.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/AMBALALSA_BO.csv  saved.
Get data for:  HDFC.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/HDFC_BO.csv  saved.
Get data for:  ANDHRAPET.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/ANDHRAPET_BO.csv  saved.
Get data for:  ANSALAPI.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/ANSALAPI_BO.csv  saved.
Get data for:  UTIQUE.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/UTIQUE_BO.csv  saved.
Get data for:  ARUNAHTEL.BO
/Users/iamsagar/Desktop/Trading Project/data/historical_all/ARUNAHTEL_BO.csv  saved.
Get data for: